# [Module 8] Recommendation API 호출 

# 0. 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [4]:
%store -r

# 1. 아이템 테이터 로딩

- 추천 결과의 ITEM_ID 의 상세 내용을 확인하기 위해서 로딩 함.

In [5]:
from src.p_utils import get_item_list_details

items_df = pd.read_csv(base_items_filename)

items_df.head()

,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,e1669081-8ffc-4dec-97a6-e9176d7f6651,Sans Pareil Scarf,apparel,scarf,Sans pareil scarf for women,124.99
1,cfafd627-7d6b-43a5-be05-4c7937be417d,Chef Knife,housewares,kitchen,A must-have for your kitchen,57.99
2,6e6ad102-7510-4a02-b8ce-5a0cd6f431d1,Gainsboro Jacket,apparel,jacket,This gainsboro jacket for women is perfect for...,133.99
3,49b89871-5fe7-4898-b99d-953e15fb42b2,High Definition Speakers,electronics,speaker,High definition speakers to fill the house wit...,196.99
4,5cb18925-3a3c-4867-8f1c-46efd7eba067,Spiffy Sandals,footwear,sandals,This spiffy pair of sandals for woman is perfe...,9.99


# 2. Recommendation API 호출

- 두 명의 user_id (에: 1, 100) 에 대해서 추천 결과를 얻음.

### 2-1. 사용자 과거 Interactions 확인 (user_id = 1)

In [6]:
from datetime import datetime


user_id = 1


def get_user_interaction_history(interactions_df, items_df, user_id):
    # 특정 사용자의 상호작용 기록 필터링
    user_interactions = interactions_df[interactions_df['USER_ID'] == user_id]
    
    # 시간순으로 정렬
    if 'TIMESTAMP' in user_interactions.columns:
        user_interactions = user_interactions.sort_values('TIMESTAMP', ascending=False)
    
    if len(user_interactions) == 0:
        print("상호작용 기록이 없습니다.")
        return
    
    # 결과를 저장할 리스트
    interaction_records = []
    
    # 각 상호작용에 대해 정보 수집
    for _, row in user_interactions.iterrows():
        item_info = items_df[items_df['ITEM_ID'] == row['ITEM_ID']]
        
        if not item_info.empty:
            # Unix timestamp를 datetime으로 변환
            timestamp = datetime.fromtimestamp(row['TIMESTAMP']).strftime('%Y-%m-%d %H:%M:%S')
            
            interaction_records.append({
                'TIMESTAMP': timestamp,
                'NAME': item_info['NAME'].iloc[0],
                'CATEGORY': item_info['CATEGORY_L1'].iloc[0],
                'STYLE': item_info['STYLE'].iloc[0],
                'PRICE': f"${item_info['PRICE'].iloc[0]}",
                'EVENT_TYPE': row['EVENT_TYPE']
            })
    
    # DataFrame 생성
    result_df = pd.DataFrame(interaction_records)
    
    # 중복 제거
    result_df = result_df.drop_duplicates()
    
    # 컬럼 순서 지정
    columns_order = ['TIMESTAMP', 'NAME', 'CATEGORY', 'STYLE', 'PRICE', 'EVENT_TYPE']
    result_df = result_df[columns_order]
    
    return result_df


print(f"\nUser {user_id}의 과거 상호작용 기록:")
print("----------------------------------------")
interaction_history_df = get_user_interaction_history(interactions_df, items_df, user_id)
print(interaction_history_df.to_string(index=False))


User 1의 과거 상호작용 기록:
----------------------------------------
          TIMESTAMP                      NAME   CATEGORY      STYLE   PRICE EVENT_TYPE
2020-08-22 17:28:52 Saddle Brown Office Chair  furniture     chairs $210.99       View
2020-08-21 00:36:17                Table Lamp  homedecor   lighting  $26.99       View
2020-08-20 11:37:01        Black Office Chair  furniture     chairs $205.99       View
2020-08-20 10:19:35             Easter Basket  homedecor decorative  $18.99       View
2020-08-17 20:50:21        Black Office Chair  furniture     chairs $205.99       View
2020-08-16 20:20:45        White Dining Table  furniture     tables $225.99       View
2020-08-16 19:54:20        White Dining Table  furniture     tables $225.99       View
2020-08-16 08:32:36   Light Gray Dining Chair  furniture     chairs $120.99       View
2020-08-16 08:17:39                    Kettle housewares    kitchen  $53.99       View
2020-08-16 08:17:36                    Kettle housewares    kitchen 

### 2.2 USER-PERSONALIZATION 솔루션 추천 생성 : user_id = 1

In [7]:
user_id = 1

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = base_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")    
print("Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  1
-------------------
Solution Recommendation


,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,557bb55d-3cac-4c5c-bb30-1d66bc60e8ae,Rosy-Brown Work Table,furniture,tables,Increase your productivity with this outstandi...,188.99
1,5763e87b-9b9c-456d-8986-1f1e34fa9318,Table Lamp,homedecor,lighting,This table lamp will enhance your home,26.99
2,328a7f6b-238d-4165-9949-5e0aa70b7200,Light Gray Dining Chair,furniture,chairs,Unparalleled light gray chair for any dining room,120.99
3,96d887df-efe3-4d05-8df0-8cb2a0674711,Jar Candle,homedecor,lighting,This jar candle will enhance your home,40.99
4,c54d3f55-35b8-47c6-9771-a573aa00f4ad,Peru-Orange Chair,furniture,chairs,Enjoy this soft peru-orange dining chair,146.99
5,c8312844-248a-4a07-96a1-4c02cb1e2471,Tan Dining Table,furniture,tables,This definitive tan dining table will delight ...,169.99
6,3ef07621-8af8-42c5-891b-94f403787fd7,Gray Work Table,furniture,tables,Increase your productivity with this outstandi...,222.99
7,d055f31f-2e3e-4338-ac64-521f88779b50,Dark Gray Worktable,furniture,tables,Unsurpassed dark gray worktable for your office,178.99
8,9747c3ce-b89d-4992-bf3d-f4ecc5a668fe,Dark Gray Office Chair,furniture,chairs,Unsurpassed dark gray chair for your office,211.99
9,ff47435d-97d2-4d05-b3e8-294d4c47cbc3,Dark Gray Worktable,furniture,tables,Unrivaled dark gray worktable for your office,212.99


### 2.3 사용자 과거 Interactions 확인 (user_id = 100)

In [8]:
from datetime import datetime


user_id = 100


def get_user_interaction_history(interactions_df, items_df, user_id):
    # 특정 사용자의 상호작용 기록 필터링
    user_interactions = interactions_df[interactions_df['USER_ID'] == user_id]
    
    # 시간순으로 정렬
    if 'TIMESTAMP' in user_interactions.columns:
        user_interactions = user_interactions.sort_values('TIMESTAMP', ascending=False)
    
    if len(user_interactions) == 0:
        print("상호작용 기록이 없습니다.")
        return
    
    # 결과를 저장할 리스트
    interaction_records = []
    
    # 각 상호작용에 대해 정보 수집
    for _, row in user_interactions.iterrows():
        item_info = items_df[items_df['ITEM_ID'] == row['ITEM_ID']]
        
        if not item_info.empty:
            # Unix timestamp를 datetime으로 변환
            timestamp = datetime.fromtimestamp(row['TIMESTAMP']).strftime('%Y-%m-%d %H:%M:%S')
            
            interaction_records.append({
                'TIMESTAMP': timestamp,
                'NAME': item_info['NAME'].iloc[0],
                'CATEGORY': item_info['CATEGORY_L1'].iloc[0],
                'STYLE': item_info['STYLE'].iloc[0],
                'PRICE': f"${item_info['PRICE'].iloc[0]}",
                'EVENT_TYPE': row['EVENT_TYPE']
            })
    
    # DataFrame 생성
    result_df = pd.DataFrame(interaction_records)
    
    # 중복 제거
    result_df = result_df.drop_duplicates()
    
    # 컬럼 순서 지정
    columns_order = ['TIMESTAMP', 'NAME', 'CATEGORY', 'STYLE', 'PRICE', 'EVENT_TYPE']
    result_df = result_df[columns_order]
    
    return result_df


print(f"\nUser {user_id}의 과거 상호작용 기록:")
print("----------------------------------------")
interaction_history_df = get_user_interaction_history(interactions_df, items_df, user_id)
print(interaction_history_df.to_string(index=False))


User 100의 과거 상호작용 기록:
----------------------------------------
          TIMESTAMP                NAME    CATEGORY      STYLE   PRICE EVENT_TYPE
2020-08-23 09:50:21 Accurate Microphone instruments microphone $499.99       View
2020-08-23 09:50:20 Accurate Microphone instruments microphone $499.99       View
2020-08-20 09:02:08         French Horn instruments       wind $192.99       View
2020-08-17 15:54:32           Saxophone instruments       wind  $95.99       View
2020-08-12 02:50:13     Electric Guitar instruments    strings $114.99       View
2020-08-11 21:08:54         Visit China       books     travel  $14.99       View
2020-08-11 04:32:15     Electric Guitar instruments    strings $114.99       View
2020-08-06 22:22:40       Electric Bass instruments    strings  $57.99       View
2020-08-04 15:09:32 Accurate Microphone instruments microphone $499.99       View
2020-08-02 14:54:09           Saxophone instruments       wind  $95.99       View
2020-08-01 13:58:56 Accurate Micro

### 2.4 USER-PERSONALIZATION 솔루션 추천 생성 : user_id = 100

In [9]:
user_id = 100

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = base_campaign_arn,
    userId = str(user_id),
    numResults = 10
)


items_list = []    
for item in get_recommendations_response['itemList']:
    items_list.append(item['itemId'])   
    # print (item['itemId'])
    
print("-------------------")    
print("user_id: ", user_id)    
print("-------------------")    
print("Solution Recommendation")
get_item_list_details(items_df, items_list)  # 유의: 기존의 추천 순서와 리턴의 값이 순서가 다름


-------------------
user_id:  100
-------------------
Solution Recommendation


,ITEM_ID,NAME,CATEGORY_L1,STYLE,PRODUCT_DESCRIPTION,PRICE
0,6ca2a410-aba9-4c73-9dae-d188412e0e50,Electric Guitar,instruments,strings,This electric guitar will delight the most dem...,114.99
1,387f55e6-0eb9-47d6-9ef1-56a80b884430,Electric Bass,instruments,strings,This electric bass will delight the most deman...,57.99
2,e09d3267-1a87-4518-9fa7-fa9ed847c80d,Accurate Microphone,instruments,microphone,This accurate microphone can be relied upon in...,499.99
3,76fa669b-1611-4f31-8377-55c3e701ced4,Saxophone,instruments,wind,This saxophone is perfect for amateur and prof...,95.99
4,ab5ea156-d41d-43bc-ae5f-10b47f8e42f1,Acoustic Bass,instruments,strings,This acoustic bass is perfect for amateur and ...,187.99
5,93a1db77-615e-49aa-9f1e-edcf7c6675f4,Violin,instruments,strings,This violin is perfect for amateur and profess...,77.99
6,a54fbc82-c85d-4d21-b907-e8cdbbff69d2,Acoustic Bass,instruments,strings,This acoustic bass will delight the most deman...,221.99
7,24a19516-a629-4703-af46-cf28f604a48a,Electric Guitar,instruments,strings,This electric guitar is perfect for amateur an...,99.99
8,51fd504d-ca0a-422b-8cb4-0ca97b35b830,Acoustic Guitar,instruments,strings,This acoustic guitar is perfect for amateur an...,150.99
9,97b25afe-68ca-445d-a276-dc7e09df575d,Upright Piano,instruments,keys,This upright piano will delight the most deman...,184.99
